### Liraz Shushan , Elior Dahan 



In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

In [2]:
url_site = 'https://www.ad.co.il/nadlanrent?sp275=17413&sp277=17804,18094'
#allredy filterd by city and neighborhood

In [3]:
def get_all_apartment_urls(base_url, start_page=1, max_pages=10): 
    '''
    Fetches all apartment URLs from the given base URL and number of pages.
    '''
    
    apartment_list = []

    for page in range(start_page, max_pages + 1):
        print(f"Fetching page {page}...")
        
        url = f"{base_url}&pageindex={page}"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Stopped at page {page} (status {response.status_code})")
            break

        results_page = BeautifulSoup(response.content, 'html.parser')
        main_content = results_page.find('main') or results_page

        apartments = main_content.find_all('div', {'class': 'card-body p-md-3'})
        if not apartments:
            print(f"No apartments found on page {page}. Stopping.")
            break
        
        for apartment in apartments:
            href = apartment.find('a')
            if href:
                apartment_link = "https://www.ad.co.il" + href.get('href')
                apartment_list.append(apartment_link)

    return apartment_list


In [4]:
url_list = get_all_apartment_urls(url_site)
print(f"we have {len(url_list)} apartment.")


Fetching page 1...
Fetching page 2...
Fetching page 3...
No apartments found on page 3. Stopping.
we have 58 apartment.


In [5]:
features_from_table = [
    "פרטי הנכס",
    "שכונה",
    "כתובת",
    "חדרים",
    "קומה",
    "שטח בנוי",
    "שטח גינה",
    "תאריך כניסה",
    "תשלומים בשנה",
    "ארנונה בחודש",
    "ועד בית בחודש"
]

features_from_icons = [
    "מרוהטת",
    "מזגן",
    "חניה",
    "ממ\"ד",
    "מרפסת",
    "נגישות",
    "סורגים",
    "מעלית",
    "מחסן",
    "משופצת"
]

def extract_apartment_details(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error fetching {url}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')
    data = {}

    # -------- features from table--------
    table = soup.find('table')
    table_data = {}
    if table:
        for row in table.find_all('tr'):
            cells = row.find_all('td')
            if len(cells) == 2:
                key = cells[0].get_text(strip=True)
                value = cells[1].get_text(strip=True)
                table_data[key] = value

    for field in features_from_table:
        data[field] = table_data.get(field, None)
    
    # -------- features from icons --------
    feature_section = soup.find('div', class_='card-icons')
    features_data = {feature: 0 for feature in features_from_icons}  # default to 0

    if feature_section:
        icons = feature_section.find_all('div', class_='card-icon')
        for icon in icons:
            feature_name = icon.find('span').get_text(strip=True)
            has_feature = icon.find('i', class_='fa-check') is not None
            if feature_name in features_data:
                features_data[feature_name] = int(has_feature)

    data.update(features_data) 
 
    # -------- more features --------
    discription = soup.find('p', class_='text-word-break')
    data['תיאור'] = discription.get_text(strip=True) if discription else None
    price = soup.find('div', class_='d-flex justify-content-between').find_all('h2')[1].text.strip()
    data['מחיר'] = int(price.replace('₪', '').replace(',', '').strip())
    data['מספר תמונות'] = len(soup.find_all('a', class_='post-1'))
    

    return data


In [6]:
results = []

for url in url_list:
    details = extract_apartment_details(url)
    if details:
        results.append(details)
    else:
        print(f"Failed to extract details from {url}")

df = pd.DataFrame(results)

#df

In [ ]:
def get_distance(location1, location2):
    url = f"https://routes.googleapis.com/directions/v2:computeRoutes"

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': "AIzaSyAM5c......", #### replace with your key
        'X-Goog-FieldMask': 'routes.distanceMeters'
    }

    body = {
        "origin": {
            "address": location1
        },
        "destination": {
            "address": location2
        },
        "travelMode": "DRIVE",  # BICYCLE / WALK / TWO_WHEELER / TRANSIT
        "routingPreference": "TRAFFIC_AWARE"
    }
    response = requests.post(url, headers=headers, data=json.dumps(body))

    try:
        if response.status_code == 200:
            try:
                data = response.json()
                distance_meters = data['routes'][0]['distanceMeters']
                return distance_meters
            except:
                print("Response not in valid JSON format")
        else:
            print("Error:", response.status_code, response.text)
            return None
    except:
        print("Something went wrong with requests.get")

In [8]:
def compute_distances(df):
    distances = []

    for idx, row in df.iterrows():
        if pd.notna(row["כתובת"]):
            full_address = f"{row['כתובת']}, תל אביב"
        elif pd.notna(row["שכונה"]):
            full_address = f"{row['שכונה']}, תל אביב"
        else:
            distances.append(None)
            continue

        result = get_distance("כיכר דיזינגוף, תל אביב", full_address)
        if result:
            distances.append(result / 1000)  # Convert to km
        else:
            distances.append(None)

    df["distance_from_Dizengoff"] = distances
    return df




In [9]:
compute_distances(df)

,פרטי הנכס,שכונה,כתובת,חדרים,קומה,שטח בנוי,שטח גינה,תאריך כניסה,תשלומים בשנה,ארנונה בחודש,...,מרפסת,נגישות,סורגים,מעלית,מחסן,משופצת,תיאור,מחיר,מספר תמונות,distance_from_Dizengoff
0,דירה,נוה אביבים,יהודה הנשיא 18,4,3 מתוך 4,95,None,מיידית,12,515,...,1,0,0,1,0,0,פירוט ריהוטחלקי מיטות וארונות ספה,6600,0,6.132
1,דירה,הצפון החדש החלק הדרומי,דפנה 28,4,3 מתוך 3,100,None,מיידית,12,350,...,0,0,0,0,0,0,"ללא תיווך! מפרטי! להשכרה דירת 4 חד׳ גדולה, \n...",8450,9,3.014
2,כללי,הצפון החדש החלק הדרומי,אבן גבירול 114,1,קרקע מתוך קרקע,38,None,מיידית,12,None,...,0,0,0,0,0,0,ללא תיווך חנות קטנה ומטריפה פינתית באיבן גבירו...,111,3,1.430
3,כללי,הצפון החדש החלק הדרומי,אבן גבירול 114,1,קרקע מתוך קרקע,38,None,מיידית,12,None,...,0,0,1,0,0,1,חנות קטנה ומטריפה להשכרה 38 מ 21 מ וגלריה 17 מ...,1,2,1.430
4,דירה,הצפון החדש החלק הדרומי,ויצמן 2,4,1 מתוך 4,110,None,מיידית,12,400,...,1,0,0,0,0,1,"דירה שמורה, צבועה, במצב טוב לאחר שיפוץ. \nמיקו...",10500,13,1.888
5,מרתף/פרטר,הצפון החדש החלק הדרומי,אורי 14,3,קרקע מתוך 4,65,None,מיידית,12,275,...,0,0,1,1,0,0,"שקט ונעים ממש במרכז העיר, דירת פרטר 65מ""ר עם ח...",67000,1,1.742
6,דירה,נוה אביבים,מוריץ דניאל 36,3,8 מתוך 16,100,None,None,12,600,...,1,1,0,1,0,0,4 חדרים שהוסבו ל-3 חדרים סלון גדול\nצעד מאוניב...,9000,7,7.658
7,דירה,הצפון החדש החלק הדרומי,דובנוב 30,3,4 מתוך 8,92,None,מיידית,12,675,...,1,0,0,1,1,0,מפרטי. דירת יוקרה בקומה רביעית ונוף פתוח. חדשה...,13500,10,1.611
8,דופלקס,נוה אביבים,הופיין 3,3.5,קרקע מתוך 2,107,None,None,12,361,...,0,0,1,0,0,1,"דירה ייחודית שאין כמוה, מוזמנים לבוא ולהתאהב.\...",11000,5,7.774
9,דירה,נוה אביבים,הופיין 8,2.5,3 מתוך 3,59,None,מיידית,12,200,...,0,0,0,0,0,1,"רמת אביב הירוקה\nדירה של 2.5 חדרים,כ - 59 מטר ...",5600,8,7.827


Arranging the data frame and fit values and data types

In [10]:
df["קומות בבניין"] = 0
#Columns order
columns_in_order = [
    "פרטי הנכס", "שכונה", "כתובת", "חדרים", "קומה", "שטח בנוי", "שטח גינה", "תאריך כניסה",
    "תשלומים בשנה", "ארנונה בחודש", "ועד בית בחודש", "קומות בבניין", "תיאור", "חניה", "מחסן",
    "מעלית", "מזגן", "נגישות", "סורגים", "ממ\"ד", "מרפסת", "מרוהטת", "משופצת", "מחיר", "מספר תמונות", "distance_from_Dizengoff"
]

df = df[columns_in_order]


In [11]:
#Updating column names
column_rename_map = {
    "פרטי הנכס": "property_type",
    "שכונה": "neighborhood",
    "כתובת": "address",
    "חדרים": "room_num",
    "קומה": "floor",
    "שטח בנוי": "area",
    "שטח גינה": "garden_area",
    "תאריך כניסה": "days_to_enter",
    "תשלומים בשנה": "num_of_payments",
    "ארנונה בחודש": "monthly_arnona",
    "ועד בית בחודש": "building_tax",
    "קומות בבניין": "total_floors",
    "תיאור": "description",
    "חניה": "has_parking",
    "מחסן": "has_stotsge",
    "מעלית": "elevator",
    "מזגן": "ac",
    "נגישות": "handicap",
    "סורגים": "has_bars",
    "ממ\"ד": "has_safe_room",
    "מרפסת": "has_balcon",
    "מרוהטת": "is_furnished",
    "משופצת": "is_renovated",
    "מחיר": "price",
    "מספר תמונות": "num_of_images",
    "distance_from_Dizengoff": "distance_from_center"
}

df.rename(columns=column_rename_map, inplace=True)


In [12]:
# example: "קרקע מתוך 5" -> "0 מתוך 5"
df["floor"] = df["floor"].str.replace("קרקע", "0", regex=False)

df["total_floors"] = df["floor"].str.split(" מתוך ").str[1]
df["total_floors"] = pd.to_numeric(df["total_floors"], errors='coerce')

df["floor"] = df["floor"].str.split(" מתוך ").str[0]
df["floor"] = pd.to_numeric(df["floor"], errors='coerce')

df["garden_area"] = df["garden_area"].apply(lambda x: 0 if x is None else x)

df["days_to_enter"] = df["days_to_enter"].replace("מיידית", 0)


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   property_type         58 non-null     object 
 1   neighborhood          58 non-null     object 
 2   address               47 non-null     object 
 3   room_num              58 non-null     object 
 4   floor                 58 non-null     int64  
 5   area                  58 non-null     object 
 6   garden_area           58 non-null     int64  
 7   days_to_enter         43 non-null     float64
 8   num_of_payments       58 non-null     object 
 9   monthly_arnona        47 non-null     object 
 10  building_tax          46 non-null     object 
 11  total_floors          58 non-null     int64  
 12  description           57 non-null     object 
 13  has_parking           58 non-null     int64  
 14  has_stotsge           58 non-null     int64  
 15  elevator              58 

In [14]:
df['property_type'] = df['property_type'].astype('string')
df['neighborhood'] = df['neighborhood'].astype('string')
df['address'] = df['address'].astype('string')
df['room_num'] = df['room_num'].astype('float')
df['floor'] = df['floor'].astype('Int64')
df['area'] = df['area'].astype('Int64')
df['garden_area'] = df['garden_area'].astype('Int64')
df['days_to_enter'] = df['days_to_enter'].astype('Int64')
df['num_of_payments'] = df['num_of_payments'].astype('Int64')
df['monthly_arnona'] = df['monthly_arnona'].astype('Int64')
df['building_tax'] = df['building_tax'].astype('Int64')
df['total_floors'] = df['total_floors'].astype('Int64')
df['description'] = df['description'].astype('string')
df['has_parking'] = df['has_parking'].astype('Int64')
df['has_stotsge'] = df['has_stotsge'].astype('Int64')
df['elevator'] = df['elevator'].astype('Int64')
df['ac'] = df['ac'].astype('Int64')
df['handicap'] = df['handicap'].astype('Int64')
df['has_bars'] = df['has_bars'].astype('Int64')
df['has_safe_room'] = df['has_safe_room'].astype('Int64')
df['has_balcon'] = df['has_balcon'].astype('Int64')
df['is_furnished'] = df['is_furnished'].astype('Int64')
df['is_renovated'] = df['is_renovated'].astype('Int64')
df['price'] = df['price'].astype('float')
df['num_of_images'] = df['num_of_images'].astype('Int64')
df['distance_from_center'] = df['distance_from_center'].astype('float')

In [16]:
df.to_csv('apartments.csv', index=False, encoding='utf-8-sig')